In [1]:
from keras.models import load_model

In [2]:
import numpy as np
import pandas as pd
from numpy import array
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.express as px
import dash_core_components as dcc


In [3]:
model = load_model('model-089.model')

In [4]:
df=pd.read_csv('NSE-Tata-Global-Beverages-Limited.csv')

In [5]:
df=df.sort_values(by=['Date'])

In [6]:
df1=df.reset_index()['Close']

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [8]:
##splitting dataset into train and test split
training_size=int(len(df1)*0.7)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [9]:
x_input=test_data[(len(test_data) -100):].reshape(1,-1)
x_input1=x_input.copy()

In [10]:
temp_input=list(x_input1)
temp_input=temp_input[0].tolist()
temp_input1=temp_input.copy()

In [11]:
def get_prediction(x_input,temp_input,model,n_days):
    lst_output=[]
    n_steps=x_input.shape[1]
    temp1=temp_input.copy()
    x_input1=x_input.copy()
    i=0
    while(i<n_days):
    
        if(len(temp_input)>100):
            #print(temp_input)
            x_input=np.array(temp_input[1:])
            print("{} day input {}".format(i,x_input))
            x_input=x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            #print(x_input)
            yhat = model.predict(x_input, verbose=0)
            print("{} day output {}".format(i,yhat))
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.extend(yhat.tolist())
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input, verbose=0)
            print(yhat[0])
            temp_input.extend(yhat[0].tolist())
            print(len(temp_input))
            lst_output.extend(yhat.tolist())
            i=i+1


    #return df_pred
    return temp_input,lst_output

In [12]:
def create_data(x_input,lst_output,n_days):
    day_new=np.arange(1,(x_input.shape[1]+1))
    day_pred=np.arange((x_input.shape[1]+1),((x_input.shape[1]+1)+n_days))
    df_dt=df[['Date','Close']]
    df_dt['Date']= pd.to_datetime(df_dt['Date'])
    dates=[]
    for i in range(1,n_days+1):
        d=(df_dt["Date"].iloc[-1] + pd.DateOffset(days=i))
        f=str(d).split()[0]
        dates.append(f)
    predicted_value=[]
    for i in scaler.inverse_transform(lst_output):
        predicted_value.append(i[0])
    df_pred = pd.DataFrame(list(zip(dates, predicted_value)),columns =['Date', 'Predicted_Stock _value'])
    df_pred.index = pd.to_datetime(df_pred['Date'])
    return df_pred


In [13]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
import dash_table

In [15]:
app = dash.Dash(__name__)

# Define the app
app.layout = html.Div(children=[
                      html.Div(className='row',  # Define the row element
                               children=[
                                  html.Div(className='four columns div-user-controls',
                                          children=[
                                              html.H2('Stock Price'),
                                              html.P('''Predecting the stock price'''),
                                              dcc.Dropdown(
                                              id='day_number_dropdown',
                                                  options=[
                                                      {'label': 'Next  week', 'value': '7'},
                                                      {'label': 'Next 15 days', 'value': '15'},
                                                      {'label': 'Next 5 days', 'value': '5'},
                                                      {'label': 'next month', 'value': '30'}
                                                  ],
                                                  value='5'
                                              ),
                                              html.Div(id='dropdown_output')
                                          ]),  # Define the left element
                                  html.Div(className='eight columns div-for-charts bg-grey',
                                          children=[
                                              dcc.Graph(id='timeseries',config={'displayModeBar': False}),
                                              

                                          ])  # Define the right element
                                  ])
                                ])

@app.callback(
    dash.dependencies.Output('timeseries', 'figure'),
    [dash.dependencies.Input('day_number_dropdown', 'value')]
)
def update_output(value):
    trace=[]
    temp_input=list(x_input1)
    temp_input=temp_input[0].tolist()
    temp_input,lst_output=get_prediction(x_input,temp_input,model,int(value))
    df_pred=create_data(x_input,lst_output,int(value))
    df_pred.index = pd.to_datetime(df_pred['Date'])
    

        
    figure=px.line(df_pred,
                   x='Date',
                   y='Predicted_Stock _value',
                   template='plotly_dark')
    
    return figure
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
[0.49636334]
101
1 day input [0.62057373 0.64926042 0.63671    0.63155536 0.64769162 0.66136262
 0.68063649 0.69565217 0.7082026  0.69408337 0.72545944 0.74652622
 0.71985657 0.68399821 0.72209771 0.7310623  0.7371134  0.73644106
 0.7218736  0.73935455 0.74831914 0.78507396 0.75974899 0.77409234
 0.73688929 0.70909906 0.69722098 0.71470193 0.72545944 0.73823398
 0.73038996 0.72299417 0.74719857 0.72523532 0.7994173  0.78283281
 0.75571493 0.73352757 0.77364411 0.77140296 0.7492156  0.72859704
 0.70035858 0.63021067 0.65105334 0.60959211 0.58471537 0.609368
 0.63043478 0.64388167 0.6017481  0.61160914 0.62572837 0.62550426
 0.64657104 0.59614523 0.59524877 0.59166293 0.61340206 0.6048857
 0.59928283 0.60354101 0.58673241 0.5912147  0.60466159 0.61272972
 0.62909009 0.6387

C:\Users\MYT\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[0.49636334]
101
1 day input [0.62057373 0.64926042 0.63671    0.63155536 0.64769162 0.66136262
 0.68063649 0.69565217 0.7082026  0.69408337 0.72545944 0.74652622
 0.71985657 0.68399821 0.72209771 0.7310623  0.7371134  0.73644106
 0.7218736  0.73935455 0.74831914 0.78507396 0.75974899 0.77409234
 0.73688929 0.70909906 0.69722098 0.71470193 0.72545944 0.73823398
 0.73038996 0.72299417 0.74719857 0.72523532 0.7994173  0.78283281
 0.75571493 0.73352757 0.77364411 0.77140296 0.7492156  0.72859704
 0.70035858 0.63021067 0.65105334 0.60959211 0.58471537 0.609368
 0.63043478 0.64388167 0.6017481  0.61160914 0.62572837 0.62550426
 0.64657104 0.59614523 0.59524877 0.59166293 0.61340206 0.6048857
 0.59928283 0.60354101 0.58673241 0.5912147  0.60466159 0.61272972
 0.62909009 0.63872703 0.61810847 0.59524877 0.57530255 0.60242044
 0.58583595 0.5912147  0.59771403 0.59009413 0.60085164 0.5425818
 0.53675482 0.53070372 0.53922008 0.5349619  0.50806813 0.53787539
 0.58852532 0.60040341 0.59345585 0.5

C:\Users\MYT\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[0.49636334]
101
1 day input [0.62057373 0.64926042 0.63671    0.63155536 0.64769162 0.66136262
 0.68063649 0.69565217 0.7082026  0.69408337 0.72545944 0.74652622
 0.71985657 0.68399821 0.72209771 0.7310623  0.7371134  0.73644106
 0.7218736  0.73935455 0.74831914 0.78507396 0.75974899 0.77409234
 0.73688929 0.70909906 0.69722098 0.71470193 0.72545944 0.73823398
 0.73038996 0.72299417 0.74719857 0.72523532 0.7994173  0.78283281
 0.75571493 0.73352757 0.77364411 0.77140296 0.7492156  0.72859704
 0.70035858 0.63021067 0.65105334 0.60959211 0.58471537 0.609368
 0.63043478 0.64388167 0.6017481  0.61160914 0.62572837 0.62550426
 0.64657104 0.59614523 0.59524877 0.59166293 0.61340206 0.6048857
 0.59928283 0.60354101 0.58673241 0.5912147  0.60466159 0.61272972
 0.62909009 0.63872703 0.61810847 0.59524877 0.57530255 0.60242044
 0.58583595 0.5912147  0.59771403 0.59009413 0.60085164 0.5425818
 0.53675482 0.53070372 0.53922008 0.5349619  0.50806813 0.53787539
 0.58852532 0.60040341 0.59345585 0.5

...

18 day output [[0.5108694]]
19 day input [0.7218736  0.73935455 0.74831914 0.78507396 0.75974899 0.77409234
 0.73688929 0.70909906 0.69722098 0.71470193 0.72545944 0.73823398
 0.73038996 0.72299417 0.74719857 0.72523532 0.7994173  0.78283281
 0.75571493 0.73352757 0.77364411 0.77140296 0.7492156  0.72859704
 0.70035858 0.63021067 0.65105334 0.60959211 0.58471537 0.609368
 0.63043478 0.64388167 0.6017481  0.61160914 0.62572837 0.62550426
 0.64657104 0.59614523 0.59524877 0.59166293 0.61340206 0.6048857
 0.59928283 0.60354101 0.58673241 0.5912147  0.60466159 0.61272972
 0.62909009 0.63872703 0.61810847 0.59524877 0.57530255 0.60242044
 0.58583595 0.5912147  0.59771403 0.59009413 0.60085164 0.5425818
 0.53675482 0.53070372 0.53922008 0.5349619  0.50806813 0.53787539
 0.58852532 0.60040341 0.59345585 0.59278351 0.59143882 0.58561183
 0.59816226 0.58987001 0.58538772 0.58762887 0.57485433 0.56006275
 0.51792918 0.47758853 0.50425818 0.49636334 0.5165813  0.53393394
 0.54362947 0.54318064 0.

 0.51520056 0.51533729 0.51396948 0.5114823 ]
26 day output [[0.5085946]]
27 day input [0.69722098 0.71470193 0.72545944 0.73823398 0.73038996 0.72299417
 0.74719857 0.72523532 0.7994173  0.78283281 0.75571493 0.73352757
 0.77364411 0.77140296 0.7492156  0.72859704 0.70035858 0.63021067
 0.65105334 0.60959211 0.58471537 0.609368   0.63043478 0.64388167
 0.6017481  0.61160914 0.62572837 0.62550426 0.64657104 0.59614523
 0.59524877 0.59166293 0.61340206 0.6048857  0.59928283 0.60354101
 0.58673241 0.5912147  0.60466159 0.61272972 0.62909009 0.63872703
 0.61810847 0.59524877 0.57530255 0.60242044 0.58583595 0.5912147
 0.59771403 0.59009413 0.60085164 0.5425818  0.53675482 0.53070372
 0.53922008 0.5349619  0.50806813 0.53787539 0.58852532 0.60040341
 0.59345585 0.59278351 0.59143882 0.58561183 0.59816226 0.58987001
 0.58538772 0.58762887 0.57485433 0.56006275 0.51792918 0.47758853
 0.50425818 0.49636334 0.5165813  0.53393394 0.54362947 0.54318064
 0.53418952 0.52133769 0.51004237 0.5041555

C:\Users\MYT\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[0.49636334]
101
1 day input [0.62057373 0.64926042 0.63671    0.63155536 0.64769162 0.66136262
 0.68063649 0.69565217 0.7082026  0.69408337 0.72545944 0.74652622
 0.71985657 0.68399821 0.72209771 0.7310623  0.7371134  0.73644106
 0.7218736  0.73935455 0.74831914 0.78507396 0.75974899 0.77409234
 0.73688929 0.70909906 0.69722098 0.71470193 0.72545944 0.73823398
 0.73038996 0.72299417 0.74719857 0.72523532 0.7994173  0.78283281
 0.75571493 0.73352757 0.77364411 0.77140296 0.7492156  0.72859704
 0.70035858 0.63021067 0.65105334 0.60959211 0.58471537 0.609368
 0.63043478 0.64388167 0.6017481  0.61160914 0.62572837 0.62550426
 0.64657104 0.59614523 0.59524877 0.59166293 0.61340206 0.6048857
 0.59928283 0.60354101 0.58673241 0.5912147  0.60466159 0.61272972
 0.62909009 0.63872703 0.61810847 0.59524877 0.57530255 0.60242044
 0.58583595 0.5912147  0.59771403 0.59009413 0.60085164 0.5425818
 0.53675482 0.53070372 0.53922008 0.5349619  0.50806813 0.53787539
 0.58852532 0.60040341 0.59345585 0.5

C:\Users\MYT\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

